In [ ]:
import ultralytics
import os
from dotenv import load_dotenv
from roboflow import Roboflow
import cv2
from ultralytics import YOLO


In [ ]:
load_dotenv()
ultralytics.checks()

# Download dataset with 2096 butterflies.

In [ ]:
rf = Roboflow(api_key=os.getenv('ROBOFLOW_API_KEY'))
workspace = rf.workspace("firstwp-gzzwu")
project = workspace.project("butterfly-uajhb-ab2co")
version = project.version(3)
dataset = version.download("yolov11")

# Train it using yolo-11-s and image size of 640

In [ ]:
# !yolo task=detect mode=train model=yolo11s.pt data={dataset.location}/data.yaml epochs=10 imgsz=640 plots=True

In [ ]:
# Evaluate the model

In [ ]:
!yolo task=detect mode=val model=runs/colab/weights/best.pt data={dataset.location}/data.yaml

In [ ]:
!yolo task=detect mode=predict model=runs/colab/weights/best.pt conf=0.55 source=report_images_test save=True

In [ ]:
model_trained = YOLO("runs/colab/weights/best.pt")

In [ ]:
actual_results = []
expected_predictions = []

In [ ]:
for expected_label, directory in [(0, os.getenv('NEGATIVE_IMAGES_CHALLENGE_PATH')), (1, os.getenv('POSITIVE_IMAGES_CHALLENGE_PATH'))]:
    list_images = os.listdir(directory)
    expected_predictions.extend([expected_label] * len(list_images))
    for image_name in list_images:
        results = model_trained(os.path.join(directory, image_name), conf=0.55)
        r = [1 for k in results if len(k.boxes.cls.cpu().numpy()) > 0]
        actual_results.append(1 if len(r) > 0 else 0)

In [ ]:
from sklearn.metrics import accuracy_score, f1_score

In [ ]:
accuracy = accuracy_score(expected_predictions, actual_results)
f1 = f1_score(expected_predictions, actual_results, zero_division=0)
print(accuracy)
print(f1)

In [ ]:
accuracy = accuracy_score(expected_predictions, actual_results)
f1 = f1_score(expected_predictions, actual_results, zero_division=0)
print(accuracy)
print(f1)